In [2]:
import pandas as pd
import json
import sys
import datetime
import shutil
import os

import datetime



from google.cloud import storage, bigquery


load_dotenv() # Cargar las variables de entorno desde el archivo .env

True

# Bigquery

In [11]:
REGION = "us-central1"
PROJECT_ID = "constitucion-20210416"
BIGQUERY = "master"
TABLE = "news_eventregistry"

In [ ]:
!gcloud config set project $PROJECT_ID
!gcloud auth login # Cloud SDK
!gcloud auth application-default login

In [4]:
# Configura la conexión a BigQuery
client = bigquery.Client()

In [12]:
query = """
SELECT 
    title, 
    body, 
    date, 
    url, 
    source.title AS source_title, 
    source.dataType AS source_dataType, 
    source.uri AS source_uri, 
    time, 
    dateTime 
FROM 
    `constitucion-20210416.master.news_eventregistry` 
WHERE 
    EXTRACT(MONTH FROM date) = 7 
    AND EXTRACT(YEAR FROM date) = 2024 """

# Ejecuta la consulta
query_job = client.query(query)

In [ ]:
query_job.result()

In [14]:
df = query_job.to_dataframe()

In [15]:
df

,title,body,date,url,source_title,source_dataType,source_uri,time,dateTime
0,Revelan a cuánto asciende la fortuna de Belind...,Desde que Christian Nodal y Ángela Aguilar gri...,2024-07-01,https://www.lared.cl/2024/noticias/revelan-a-c...,La Red,news,lared.cl,16:15:22,2024-07-01 16:15:22+00:00
1,Cuatro consejos de Harvard para incluir en la ...,Los expertos en nutrición de la entidad compar...,2024-07-01,https://www.encancha.cl/enlahora/bienestar/202...,En Cancha,news,encancha.cl,17:34:57,2024-07-01 17:34:57+00:00
2,"""Nace y el doctor me dice 'no es recomendable ...","""Hoy puedo hablarlo sin llorar a moco tendido....",2024-07-01,https://glamorama.latercera.com/noticias/2024/...,Glamorama,news,glamorama.latercera.com,18:33:15,2024-07-01 18:33:15+00:00
3,"""El me quería tirar por el balcón... '¡Por fav...","""Siempre me pegaba en la cara. Es como que él ...",2024-07-01,https://glamorama.latercera.com/noticias/2024/...,Glamorama,news,glamorama.latercera.com,02:29:58,2024-07-01 02:29:58+00:00
4,"Llevaban 25 años juntos y se ""desencontraron c...","""La vida cambia"", es una verdad que mencionó J...",2024-07-01,https://glamorama.latercera.com/sin-categoria/...,Glamorama,news,glamorama.latercera.com,02:14:50,2024-07-01 02:14:50+00:00
...,...,...,...,...,...,...,...,...,...
52976,Declaran culpable a Eduardo Macaya por dos del...,Fue condenado como autor de dos delitos de abu...,2024-07-10,https://www.elrancaguino.cl/2024/07/10/declara...,El Rancagüino,news,elrancaguino.cl,19:49:58,2024-07-10 19:49:58+00:00
52977,Gran Hermano Argentina definió a su ganador: s...,Este domingo la versión trasandina de Gran Her...,2024-07-08,https://www.biobiochile.cl/noticias/espectacul...,BioBioChile,news,biobiochile.cl,20:14:09,2024-07-08 20:14:09+00:00
52978,"""El huaso Waldo"": televidentes de Gran Hermano...",El reality de CHV Gran Hermano 2 ha generado c...,2024-07-13,https://www.biobiochile.cl/noticias/espectacul...,BioBioChile,news,biobiochile.cl,16:31:38,2024-07-13 16:31:38+00:00
52979,Con el regreso de Messi al gol: Argentina venc...,La Albiceleste ganó 2-0 con goles de Julián Ál...,2024-07-10,https://www.latercera.com/el-deportivo/noticia...,LA TERCERA,news,latercera.com,10:54:21,2024-07-10 10:54:21+00:00


# Nlp

In [109]:
df = pd.read_csv('../base/noticias_temas.csv')
df = df.drop_duplicates(subset='title', keep='first')

In [ ]:
patterns = [
    'https://www.biobiochile.cl/noticias/economia/',
    'https://www.biobiochile.cl/noticias/nacional/',
    'https://www.chilevision.cl/contigo-en-la-manana/politica/',
    'https://www.df.cl/mercados/pensiones/',
    'https://www.df.cl/mercados/bolsa-monedas/',
    'https://www.df.cl/economia-y-politica/',
    'https://www.df.cl/empresas/',
    'https://cooperativa.cl/noticias/economia/',
    'https://cooperativa.cl/noticias/pais/',
    'https://www.latercera.com/politica/',
    'https://www.latercera.com/pulso/noticia/',
    'https://www.latercera.com/nacional/',
    'https://www.cnnchile.com/pais/',
    'https://www.cnnchile.com/economia/',
    'https://www.radioagricultura.cl/economia/',
    'https://www.radioagricultura.cl/nacional/',
    'https://www.radioagricultura.cl/politica/',
    'https://www.24horas.cl/actualidad/politica/',
    'https://www.eldinamo.cl/politica/',
    'https://www.eldinamo.cl/d-economia/',
    'https://www.elciudadano.com/economia/',
    'https://www.t13.cl/noticia/politica/',
    'https://www.t13.cl/noticia/elecciones-2024/',
    'https://www.t13.cl/noticia/nacional/',
    'https://www.elobservatodo.cl/noticia/economia/',
    'https://www.elobservatodo.cl/noticia/politica/',
    'https://www.elmorrocotudo.cl/noticia/economia/',
    'https://www.emol.com/noticias/Economia/',
    'https://www.elamaule.cl/noticia/economia/',
    'https://www.elamaule.cl/noticia/politica/',
    'https://www.elmostrador.cl/politica/',
    'https://www.elciudadano.com/politica/',
    'https://www.elciudadano.com/economia/'
]

df = df[df['url'].apply(lambda x: any(x.startswith(pattern) for pattern in patterns))]


In [103]:
len(df)

4121

In [ ]:
# Sea df = pd.DataFrame({ 'body': [...]})
# df_resultante = extraer_personas_y_longitud(df)

# Prefilter

In [111]:
df['title_body'] = "Titulo de la noticia:\n" + df['title'] + "\nCuerpo de la noticia:\n"  + df['body']


In [112]:
print(df['title_body'].iloc[0])


Titulo de la noticia:
[Video]   "El que nada hace nada teme": Vallejo pide no "ridiculizar" debate sobre secreto bancario
Cuerpo de la noticia:
La ministra vocera de Gobierno, Camila Vallejo, pidió no ridiculizar el debate político sobre el levantamiento del secreto bancario. La secretaria de Estado dijo que "algunos han dicho que el que nada hace nada teme, así que no debería ser una preocupación por las compras de supermercados. De verdad que llevar a ese nivel de debate es ridículo, es ridiculizar, es desestimar la inteligencia del ciudadano y la ciudadana de nuestro país".


In [113]:
df = df.sort_values(by='numero_personas', ascending=False)
df = df[df['numero_personas'] < 60]
df = df[df['numero_personas']  > 1]
len(df)

6158